In [9]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [10]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load config & data 

In [11]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [12]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/20news/instances.pkl' mode='rb' closefd=True>


In [13]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(11258, 1995)

# run

## initialize log

In [14]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [15]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < 100:
# while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   488   0  0   1029   1030  1033  0.01  0.44  0.44  0.70  0.68
1   516   1  0    955    962   962  0.01  0.46  0.45  0.68  0.67
2   540   2  0    910    927   927  0.01  0.48  0.46  0.66  0.65
3   565   3  0    887    907   906  0.02  0.48  0.46  0.69  0.68
4   604   4  0    868    889   888  0.02  0.47  0.46  0.71  0.71
5   606   5  0    850    870   870  0.03  0.46  0.46  0.71  0.70
6   615   6  0    832    850   850  0.03  0.46  0.46  0.73  0.73
7   621   7  0    819    836   836  0.03  0.45  0.46  0.74  0.73
8   578   8  0    809    827   827  0.04  0.45  0.46  0.67  0.66
9   580   9  0    803    820   821  0.04  0.44  0.46  0.67  0.66
10  585  10  0    798    815   816  0.04  0.44  0.46  0.67  0.66
11  584  11  0    794    811   811  0.05  0.44  0.46  0.67  0.66
12  591  12  0    791    808   808  0.05  0.43  0.46  0.67  0.66
13  589  13  0    789    806   806  0.05  0.43  0.46  0.65  0.65
14  633  14  0    787    805   805  0.05  0.43  0.46  0.69  0.68
15  625  15  0    785    803   803  0.05  0.43  0.46  0.68  0.67
16  619  16  0    783    801   801  0.05  0.43  0.46  0.68  0.67
17  617  17  0    783    800   800  0.06  0.43  0.46  0.68  0.67
18  621  18  0    781    799   799  0.06  0.43  0.46  0.68  0.67
19  617  19  0    781    798   798  0.06  0.43  0.46  0.68  0.67
20  649  20  0    780    798   798  0.06  0.43  0.46  0.71  0.70
21  627  21  0    781    798   797  0.06  0.43  0.46  0.68  0.67
22  625  22  0    780    797   796  0.06  0.43  0.46  0.68  0.67
23  510  23  0    780    797   796  0.06  0.43  0.46  0.68  0.66
24  660  24  0    780    797   796  0.06  0.43  0.46  0.71  0.70
25  624  25  0    780    797   796  0.06  0.42  0.46  0.68  0.66
26  624  26  0    779    796   796  0.06  0.42  0.46  0.67  0.66
27  633  27  0    778    796   795  0.06  0.42  0.46  0.67  0.66
28  670  28  0    778    795   795  0.06  0.42  0.46  0.70  0.69
29  717  29  0    777    795   794  0.06  0.42  0.46  0.67  0.66
..  ...  .. ..    ...    ...   ...   ...   ...   ...   ...   ...
57  514  57  0    768    783   781  0.07  0.40  0.47  0.69  0.67
58  499  58  0    768    783   781  0.07  0.40  0.46  0.65  0.64
59  523  59  0    768    783   781  0.07  0.40  0.46  0.69  0.67
60  547  60  0    768    783   781  0.07  0.40  0.46  0.69  0.67
61  530  61  0    769    783   781  0.07  0.40  0.46  0.65  0.64
62  550  62  0    768    783   781  0.07  0.40  0.46  0.68  0.67
63  549  63  0    767    784   781  0.07  0.40  0.46  0.68  0.67
64  532  64  0    767    784   781  0.07  0.41  0.46  0.68  0.67
65  533  65  0    768    784   781  0.07  0.41  0.46  0.68  0.67
66  532  66  0    768    784   781  0.07  0.41  0.46  0.68  0.67
67  531  67  0    768    784   781  0.07  0.41  0.46  0.68  0.67
68  549  68  0    768    784   781  0.07  0.41  0.46  0.71  0.70
69  552  69  0    768    784   781  0.07  0.41  0.46  0.71  0.70
70  539  70  0    768    784   781  0.07  0.40  0.46  0.68  0.67
71  557  71  0    766    783   781  0.07  0.41  0.46  0.71  0.70
72  557  72  0    767    783   781  0.07  0.41  0.46  0.71  0.70
73  532  73  0    766    783   781  0.07  0.41  0.47  0.68  0.67
74  539  74  0    765    782   781  0.07  0.41  0.46  0.68  0.67
75  552  75  0    765    782   781  0.07  0.41  0.47  0.71  0.69
76  692  76  0    765    781   781  0.07  0.41  0.47  0.71  0.69
77  689  77  0    765    781   780  0.07  0.41  0.47  0.71  0.69
78  690  78  0    765    781   780  0.07  0.41  0.47  0.71  0.69
79  666  79  0    764    780   779  0.07  0.41  0.47  0.68  0.66
80  670  80  0    764    779   779  0.07  0.41  0.47  0.68  0.66
81  543  81  0    764    779   779  0.07  0.41  0.47  0.68  0.66
82  565  82  0    764    779   779  0.07  0.41  0.47  0.71  0.69
83  678  83  0    764    779   779  0.07  0.41  0.47  0.68  0.66
84  716  84  0    763    778   778  0.07  0.41  0.47  0.71  0.69
85  684  85  0    763    778   778  0.07  0.41  0.

Exception ignored in: <_io.FileIO name='model/20news/ncrp/20news-mncrp-alp10.50.1-eta10-gam0.001-epoch50/log' mode='wb' closefd=True>


 0 11258 361227.0 write one article say know think people get like make
   0-1 3701 181648.0 people say go government one get make gun state think
     0-1-1 2079 8680.0 bike car ride dog dod road speed cop drive motorcycle
     0-1-2 1293 7671.0 jews jewish islam nazi islamic muslim muslims jew history german
     0-1-3 36 18.0 david every whose large concept violation nothing conversation guy care
     0-1-4 58 149.0 nsa gay homosexual respect completely free rely archive expert cipher
     0-1-5 65 21.0 director human interest setting ford population tool institute alan disease
     0-1-6 84 110.0 oo earth deserve cult larry pm kevin hall affect future
     0-1-7 86 332.0 music na communication stage fund club include generation computer teaching
   0-2 1158 89599.0 file use program window image get available write version run
     0-2-1 1113 637.0 edge road point consistent advance ibm motorcycle mi internet adapter
     0-2-2 45 14.0 kevin instruction speak test steal display poor